In [1]:
#Importing libs
import pandas as pd
import geopandas as gpd

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AeaYSHAN65khk4tdg28BVrplyliUIxSaSbXTjq7VzWIMotpex9hjCobtaxc

Successfully saved authorization token.


In [3]:
# coordinates of Nairobi
lat =  -1.286389
lon = 36.817222

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2018-1-01'

# end date
end_date = '2022-12-31'

In [4]:
# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [5]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

Total number: 107


In [6]:
# information about the first image in our collection
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 173685, 0, -30, -44085]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 173685, 0, -30, -44085]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 173685, 0, -30, -44085]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 173685, 0, -30, -44085]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    '

In [7]:
# what about cloud cover of our first image?
landsat.first().get('CLOUD_COVER').getInfo()

8.39

In [8]:
# when was the first image taken?
landsat.first().get('DATE_ACQUIRED').getInfo()

'2018-01-13'

In [12]:

landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [14]:
ee.Image(landsat.first()).getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 173685, 0, -30, -44085]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 173685, 0, -30, -44085]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 173685, 0, -30, -44085]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 173685, 0, -30, -44085]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    '

In [15]:
import matplotlib.pyplot as plt

In [16]:
from IPython.display import Image

landsat_image = ee.Image(landsat.first())

thumbnail_url = landsat_image.getThumbUrl({
#     'min': 0,
#     'max': 3000,
#     'dimensions': 512,
#     'format': 'png',
#     'bands': ['SR_B4', 'SR_B3', 'SR_B2']
    'min': 7000,
    'max': 16000,
    'dimensions': 800, # square size in pixels
#     'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
})

display(Image(url=thumbnail_url))


In [17]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

In [18]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [20]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(landsat.size().getInfo()):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # print the image info
    print('Image #',i,date,'Cloud cover:',cloud)
    
    # display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i,date,cloud]

    # append the data 
    data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 2018-01-13 Cloud cover: 8.39


Image # 1 2018-01-29 Cloud cover: 0.02


Image # 2 2018-02-14 Cloud cover: 46.11


Image # 3 2018-03-02 Cloud cover: 86.59


Image # 4 2018-03-18 Cloud cover: 63.12


...

Image # 102 2022-10-07 Cloud cover: 55.48


Image # 103 2022-10-23 Cloud cover: 27.83


Image # 104 2022-11-08 Cloud cover: 39.7


Image # 105 2022-11-24 Cloud cover: 79.47


Image # 106 2022-12-26 Cloud cover: 24.72


In [22]:
# our data in a dataframe
df.head()

,Image #,Date,Cloud Cover
0,0,2018-01-13,8.39
1,1,2018-01-29,0.02
2,2,2018-02-14,46.11
3,3,2018-03-02,86.59
4,4,2018-03-18,63.12


In [23]:
df.tail()

,Image #,Date,Cloud Cover
102,102,2022-10-07,55.48
103,103,2022-10-23,27.83
104,104,2022-11-08,39.70
105,105,2022-11-24,79.47
106,106,2022-12-26,24.72


# Selecting images, zooming in

In [25]:
# Filter the DataFrame to get Image # with Cloud Cover less than 10
landsat_sequence = df[df['Cloud Cover'] < 10]['Image #'].tolist()


In [26]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(10000) # meters

In [27]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region':roi
             }

In [28]:
for i in landsat_sequence:
    
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 0 2018-01-13 Cloud cover: 8.39


Image # 1 2018-01-29 Cloud cover: 0.02


Image # 17 2018-10-12 Cloud cover: 3.85


Image # 24 2019-02-01 Cloud cover: 7.01


Image # 27 2019-03-21 Cloud cover: 1.44


...

Image # 58 2020-08-30 Cloud cover: 3.27


Image # 88 2022-01-24 Cloud cover: 7.81


Image # 89 2022-02-09 Cloud cover: 0.41


Image # 91 2022-03-13 Cloud cover: 0.02


Image # 99 2022-08-20 Cloud cover: 0.91


# Calculating NDVI in Google Earth Engine

In [29]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [30]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

Image # 0 2018-01-13


Image # 1 2018-01-29


Image # 17 2018-10-12


Image # 24 2019-02-01


Image # 27 2019-03-21


...

Image # 58 2020-08-30


Image # 88 2022-01-24


Image # 89 2022-02-09


Image # 91 2022-03-13


Image # 99 2022-08-20


In [31]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])

In [32]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [33]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

In [35]:
#Saving the folium map as an html file
my_map.save('nairobi.html')